**Date Created: 10/19/2025**

In [439]:
import os
import json
import requests
import pandas as pd
from datetime import datetime
from mysql import connector
from dotenv import load_dotenv

In [440]:
load_dotenv() # this must be run in order to grant getenv function permission to access objects in .env file. 

True

In [441]:
# senseitive information stored in .env file
API_KEY = os.getenv("API_KEY")
API_HOST = os.getenv("API_HOST")

## GET Request (Extract)

In [442]:
# API Request
# first url --> url = f"https://api.eia.gov/v2/electricity/retail-sales/data/?api_key={API_KEY}&frequency=monthly&data[0]=customers&data[1]=price&data[2]=revenue&data[3]=sales&facets[stateid][]=CA&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000"
url = f"https://api.eia.gov/v2/electricity/retail-sales/data/?api_key={API_KEY}&frequency=annual&data[0]=customers&data[1]=price&data[2]=revenue&data[3]=sales&start=2019&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000"
# Don't forget to add "api_key =" and "&" after you insert API KEY variable
# Full documentation here: https://www.eia.gov/opendata/documentation/APIv2.1.0.pdf

In [443]:
# Storing the API response into a variable
response = requests.get(url)
# Printing response would return the 200 code
json_data = response.json() # accessing the json formatted data
json_data 
# examines what the data hierarchy looks like within the "response" key, where the data we want is imbedded

# there are a total of 1770 records

{'response': {'total': '2232',
  'dateFormat': 'YYYY',
  'frequency': 'annual',
  'data': [{'period': '2024',
    'stateid': 'AK',
    'stateDescription': 'Alaska',
    'sectorid': 'ALL',
    'sectorName': 'all sectors',
    'customers': '356597',
    'price': '22.17',
    'revenue': '1340.53299',
    'sales': '6046.50298',
    'customers-units': 'number of customers',
    'price-units': 'cents per kilowatt-hour',
    'revenue-units': 'million dollars',
    'sales-units': 'million kilowatt hours'},
   {'period': '2024',
    'stateid': 'AK',
    'stateDescription': 'Alaska',
    'sectorid': 'COM',
    'sectorName': 'commercial',
    'customers': '57009',
    'price': '21.57',
    'revenue': '561.56579',
    'sales': '2603.67399',
    'customers-units': 'number of customers',
    'price-units': 'cents per kilowatt-hour',
    'revenue-units': 'million dollars',
    'sales-units': 'million kilowatt hours'},
   {'period': '2024',
    'stateid': 'AK',
    'stateDescription': 'Alaska',
    's

In [444]:
 # json_data contains a lot of nested data, we need to specify what we want to access
records = json_data["response"]["data"] 
# records is stored as a list of dictionaries, with each dictionary being one row of data

## Transform

In [445]:
rows = [] # creating an empty list that will hold a row for each item
column_names = ['period', 'stateid', 'sectorid', 'customers', 'price', 'revenue', 'sales']

In [446]:
# loop through each record in the API response for the fields that are relevant, excluded state and unit measure descriptions
for record in records:
        period = record["period"]
        stateid = record["stateid"]
        sectorid = record["sectorid"]
        customers = record["customers"]
        price = record["price"]
        revenue = record["revenue"]
        sales = record["sales"]

        # putting each row into a tuple (instead of list, for its immutability)
        data_tuple = (period, stateid, sectorid, customers, price, revenue, sales)

        # append each row tuple into the rows list
        rows.append(data_tuple)

In [447]:
# create pandas dataframe
df = pd.DataFrame(rows, columns=column_names)
df.head(7)
# null values are represented as None or NaN (not a number), with the latter having a mathematical significance to it. Source: https://www.geeksforgeeks.org/python/difference-between-nan-and-none-in-python/


# how to force data type and skip over rows with missing data?

,period,stateid,sectorid,customers,price,revenue,sales
0,2024,AK,ALL,356597,22.17,1340.53299,6046.50298
1,2024,AK,COM,57009,21.57,561.56579,2603.67399
2,2024,AK,IND,1191,19.31,264.99921,1372.23499
3,2024,AK,OTH,None,None,None,None
4,2024,AK,RES,298397,24.82,513.968,2070.59399
5,2024,AK,TRA,0,0,0,0
6,2024,AL,ALL,2795343,11.9,10415.17361,87522.93201


In [448]:
df.info()
# why is Dtype all objects? Suspect there are misisng data causing python reader to cast entire column to object type. 
# Non-Null Count indicates that there are lots of missing data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2232 entries, 0 to 2231
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   period     2232 non-null   object
 1   stateid    2232 non-null   object
 2   sectorid   2232 non-null   object
 3   customers  1860 non-null   object
 4   price      1860 non-null   object
 5   revenue    1860 non-null   object
 6   sales      1860 non-null   object
dtypes: object(7)
memory usage: 122.2+ KB


### Detecting & Handling Nulls in Pandas Dataframe

In [449]:
# Pandas provides 2 functions to detect null values (NaN and None)
# isnull() and isna()

# using isnull(), where True represents missing data
nulls = df.isnull()
nulls.head(3)

# using isna() to see if there are more/less rows
nas = df.isna()
nas.head(3)

,period,stateid,sectorid,customers,price,revenue,sales
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False


In [450]:
# Dropping rows with at least one null value
    # df = df.dropna()
    # print(f"Nulls were dropped, {len(df)} records remain")

## Load into SQL

In [451]:
# loading in our variables from dotenv file for connecting to SQL  
MYSQL_HOST = os.getenv("MYSQL_HOST")
MYSQL_PORT = os.getenv("MYSQL_PORT")
MYSQL_USER = os.getenv("MYSQL_USER")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")
MYSQL_DATABASE = os.getenv("MYSQL_DATABASE")

In [452]:
# making the connection
server_connection = connector.connect(
    host = MYSQL_HOST,
    port = MYSQL_PORT,
    user = MYSQL_USER,
    password = MYSQL_PASSWORD,
    connection_timeout = 10, # we want this process to stop executing after 10 seconds in the case of an error.
    autocommit = False,
    raise_on_warnings = True
)

# a cursor is an structure allowing us to traverse a result set. Allows python to communicate with the database 
server_cursor = server_connection.cursor()

In [453]:
# not sure why we need to close the connections. To simply test if we can connect to server? 
server_cursor.close()
server_connection.close()

In [454]:
# connecting to database
db_connection = connector.connect(
    host = MYSQL_HOST,
    port = MYSQL_PORT,
    user = MYSQL_USER,
    password = MYSQL_PASSWORD,
    database = MYSQL_DATABASE
)

cur = db_connection.cursor(buffered=True)

In [455]:
# checking if our table actually exists in mysql
sql_table = "e_sales_null"
# this method executes the given database query
a = cur.execute(f"SELECT * FROM {sql_table}") 
b = cur.fetchall()
# printing a returns none, seems that cur.execute() will always return that
# Chat: cursor.execute() only executes the query and returns the number of rows affected, not the data itself.

In [456]:
if cur.fetchall() is None:
    raise SystemExit(f"This table {sql_table} is NOT found")
else:
    print(f"Success")

# seems to be giving None when I run fetchone because there is no existing data at first, then inserted some dummy data and ran again. Returned Success.

Success


In [457]:
# UPSERT = update and insert

table_cols = ['period', 'stateid', 'sectorid', 'customers', 'price', 'revenue', 'sales']

sales_df = df[table_cols]
print(sales_df)
# what is the purpose of this step? Didn't we already create a df earlier? 
# seems like it is a way to select custom columns we want from our original data frame

     period stateid sectorid customers  price     revenue       sales
0      2024      AK      ALL    356597  22.17  1340.53299  6046.50298
1      2024      AK      COM     57009  21.57   561.56579  2603.67399
2      2024      AK      IND      1191  19.31   264.99921  1372.23499
3      2024      AK      OTH      None   None        None        None
4      2024      AK      RES    298397  24.82     513.968  2070.59399
...     ...     ...      ...       ...    ...         ...         ...
2227   2019      WY      COM     58325   9.64    344.7123    3575.477
2228   2019      WY      IND     11294   6.73   695.29501   10338.641
2229   2019      WY      OTH      None   None        None        None
2230   2019      WY      RES    274881  11.18    318.4234  2849.37801
2231   2019      WY      TRA         0      0           0           0

[2232 rows x 7 columns]


In [458]:
# extract rows as tuples from the df
sales_tuples = sales_df.itertuples(index=False, name=None)

list_of_sales_tuples = list(sales_tuples)
# not sure what this step does, tuple format makes it easier for MySQL to read into database table?


In [459]:
UPSERT_SQL = f"""
INSERT INTO {sql_table}
(period, stateid, sectorid, customers, price, revenue, sales)
VALUES(%s, %s, %s, %s, %s, %s, %s) AS src
ON DUPLICATE KEY UPDATE
period = src.period,
stateid = src.stateid,
sectorid = src.sectorid,
customers = src.customers,
price = src.price,
revenue = src.revenue,
sales = src.sales;
"""
# why did he put triple quotes for f string? 
# ON DUPLICATE KEY UPDATE helps prevent duplicate rows, just update existing row.
# initially caused our data to be truncated because our primary key wasn't specific enough, and led to multiple records collapsing together.

In [460]:
print(rows_uploaded)

2232


In [461]:
# return how many rows we uploaded to MySQL
rows_uploaded = len(list_of_sales_tuples)
cur.execute(f"SELECT COUNT(*) FROM {sql_table}")
upload_count = cur.fetchone()[0]

try: 
    cur.executemany(UPSERT_SQL, list_of_sales_tuples)
    db_connection.commit()
    print(f"Success! Attempted to upload {rows_uploaded} records.")
    print(f"Actual records uploaded: {upload_count}")
except Exception as e:
    db_connection.rollback()
    print(f"Error! Rollback due to {e}")
finally:
    cur.close()
    db_connection.close()
    print("All database connects closed. Clean up completed.")


Success! Attempted to upload 2232 records.
Actual records uploaded: 2232
All database connects closed. Clean up completed.


### Above code chunk initially gave: Error! Rollback due to 1048 (23000): Column 'customers' cannot be null All database connects closed. Clean up completed."

potential reasons/solutions:
- make our SQL table to accept nulls
- see what our data frame currently looks like
- make sure data types are changed in our initial df table, understand how this affects later steps (what variables are using df)
- look at other articles on how to ETL, consult more sources, understand the process


Article: 
- https://www.getgalaxy.io/learn/common-errors/mysql-error-1048-column-cannot-be-null--causes-fixes
- https://levelup.gitconnected.com/rest-api-to-mysql-database-using-python-eb7b4606d6c3



## Questions:
- Did all the non null rows get loaded in? Do a before and after check.
- What if we wanted to load null values into mysql? What to do after defining schema to accept nulls? Do we need to tag null values with "\n" like in LOAD DATA INFILE()? 
- Connecting to sql server vs database? 
- Do you need to close connection everytime? Does it only upload after the connection is closed?

## Next steps:
- read more documentations on how to get pandas DF to Mysql. Common ETL problems and approaches. 